# Set up Delta Siege

## Set up Config

Allow for the specification of parameters such as:
* Number of samples used in the different phases
* Number of experiments to run
* Level of paralellism

In [ ]:
from deltasiege.utils import Config

# Set up the configuration
# n is the number of samples used in the different stages - 1.000.000 in each
n = 1_000_000
config = Config(
    n_train=n, n_init=n, n_check=n, n_final=n, 
    n_experiments=10,
    n_jobs=1,
)

## Set up Search Space

Allow for the specification of:
* The range of DP-parameters which is searched
* The inputs to the mechanism which is tested

In [ ]:
from deltasiege.ddsampler import SimpleSampler
from deltasiege.mechanism import GaussOpacusMechanism as GaussOpacusMechanism

# Basic parameters to the model
epsilon_0, delta_0, sensitivity = 0.1, 0.1, 1.0

# Use simple inputs
input_pairs = [(0.0, 1.0)]

# Set up sampler
search_space = SimpleSampler(
    GaussOpacusMechanism,
    epsilon = (epsilon_0, epsilon_0),
    delta = (delta_0, delta_0),
    sensitivity = (sensitivity, sensitivity),
    input_pairs = input_pairs
)

## Set up Classifier

Use MultiLayerPerceptron classifier as was done in the paper.``.

In [ ]:
from deltasiege.classifiers.legacy.multi_layer_perceptron import MultiLayerPerceptron
from deltasiege.classifiers.legacy.torch_optimizer_factory import AdamOptimizerFactory
from deltasiege.classifiers import *

sklearn_factory = MultiLayerPerceptron.get_factory(
    optimizer_factory=AdamOptimizerFactory(learning_rate=0.1, step_size=500),
    hidden_sizes=(),
    n_test_batches=1,
    normalize_input=False,
    epochs=10, 
    regularization_weight=0.0001,
    feature_transform=BitPatternFeatureTransformer(),
    in_dimensions=64
)

## Set up Delta Siege itself

Combine the previous setup into the DeltaSiege class and set up the folder structure

In [ ]:
from deltasiege import DeltaSiege
from deltasiege.logging import Logger

# Use deltasiege_runs as base folder to store results
# Can drop to pass a folder which makes all results to only be kept in memory
from pathlib import Path
!rm -rf deltasiege_runs
base_folder = Path("deltasiege_runs")
base_folder.mkdir(exist_ok=True, parents=True)

# Set up the search
deltasiege = DeltaSiege(
    config = config, 
    search_space = search_space, 
    classifier_factories = sklearn_factory,
    logger = Logger(Logger.State.SILENT),  # Surpresses  all outputs. Can be VERBOSE, SILENT, DOWN. 
    base_folder = base_folder,
    classifier_folder = base_folder / "shared", # Store classifier in seperate folder to share between runs
    mechanism_folder= base_folder / "shared"    # Store mechanism in seperate folder to share between runs
)

# Run different variations of Delta SIege

In [ ]:
!rm -rf deltasiege_runs

## The full Implementation

Vary both the epsilon and delta parameters while maintaning the rho-parameter constant.
This allows for a more efficent search procedure which generally yields significently better results

In [ ]:
from deltasiege.attack import WitnessOptimization, LinesearchEstimator
import numpy as np

# Use an estimator which tests over the specified epsilon and delta values
n_steps = 1000
estimator = LinesearchEstimator(
    delta_steps = np.exp(np.linspace(np.log(1e-9), 0, num=n_steps)),  # Log uniform distirbution over [10^-9, 1]
    epsilon_steps = np.linspace(0, 10, num = n_steps)                 # Uniform distribution over [0, 10]
)

# Make a factory corresponding to this search
witness = WitnessOptimization.get_factory(estimator=estimator)

# Run the search
deltasiege_run = deltasiege.run("DeltaSiege", witnesses_factories=witness)

## An Epsilon-Restricted Variant

Run the search where violations can only be established by finding a witness where
$\epsilon_0 = \epsilon_1$ and $\delta_0 < \delta_1$

In [ ]:
from deltasiege.attack import WitnessOptimization, LinesearchEstimator
import numpy as np

# Use an estimator which tests can only use epsilon = epsilon_0
estimator = LinesearchEstimator(epsilon_steps=np.array([epsilon_0]))

# Make a factory corresponding to this search
witness = WitnessOptimization.get_factory(estimator=estimator)
epsilon_fixed_run = deltasiege.run("FixedEpsilon", witnesses_factories=witness, thresh_confidence=0)

## An Delta-Restricted Variant

Run the search where violations can only be established by finding a witness where
$\epsilon_0 < \epsilon_1$ and $\delta_0 = \delta_1$

In [ ]:
from deltasiege.attack import WitnessOptimization, LinesearchEstimator
import numpy as np

# Use an estimator which tests can only use delta = delta_0
estimator = LinesearchEstimator(delta_steps=np.array([delta_0]))

# Make a factory corresponding to this search
witness = WitnessOptimization.get_factory(estimator=estimator)
delta_fixed_run = deltasiege.run("FixedDelta", witnesses_factories=witness)

## Create Figure 4 in the Paper

Combine the results of the previous results into a plot for vizualization

In [ ]:
from deltasiege.utils.plot_multiple_trials import plot_all_trials

# Plot all the results
# Can swap 90 degrees by changing x_is_delta
x_is_delta = False
fig, ax = plot_all_trials(
    (deltasiege_run, "DeltaSiege", "o", "blue"),
    (epsilon_fixed_run, "FixedDelta", "^", "orange"),
    (delta_fixed_run, "FixedEpsilon", "x", "purple"),
    x_is_delta=x_is_delta,
)

# Add ticks and labels
if x_is_delta:
    ax.set_xticks([delta_0]) 
    ax.set_xticklabels(["$\\delta_0$"], fontsize=20)
    ax.set_yticks([epsilon_0]) 
    ax.set_yticklabels(["$\\epsilon_0$"], fontsize=20)
else:
    ax.set_xticks([epsilon_0]) 
    ax.set_xticklabels(["$\\epsilon_0$"], fontsize=20)
    ax.set_yticks([delta_0]) 
    ax.set_yticklabels(["$\\delta_0$"], fontsize=20)

# Add legend and plot
fig.legend()
fig.show()

## A DP-Sniper Variant

Run the search by the method of DP-Sniper, where violations can only be established by finding a witness where
$\epsilon_0 < \epsilon_1$ and $\delta_0 = \delta_1$

In [ ]:
from deltasiege.attack import WitnessDPSniper, LinesearchEstimator
import numpy as np

# DP-Sniper seeks to establish a witness by maximizing epsilon while keeping delta fixed
estimator = LinesearchEstimator(delta_steps=np.array([delta_0]))

# Make a factory corresponding to this search
witness = WitnessDPSniper.get_factory(estimator=estimator, fixed_delta=delta_0)
dpsniper_run = deltasiege.run("DPSniper", witnesses_factories=witness)

## A DP-Opt Variant

Run the search by the method of DP-Opt, where violations can only be established by finding a witness where
$\epsilon_0 < \epsilon_1$ and $\delta_0 = \delta_1$

In [ ]:
from deltasiege.attack import WitnessDPOpt, LinesearchEstimator
import numpy as np

# DP-Opt seeks to establish a witness by maximizing epsilon while keeping delta fixed
estimator = LinesearchEstimator(delta_steps=np.array([delta_0]))

# Make a factory corresponding to this search
witness = WitnessDPOpt.get_factory(estimator=estimator, fixed_delta=delta_0)
dpopt_run = deltasiege.run("DPOpt", witnesses_factories=witness)

## A dynamic Variant

Use the Dvoretzky–Kiefer–Wolfowitz inequality, allow to only selecting the ```t``` and ```q``` parameters upon testing.

In [ ]:
from deltasiege.attack import WitnessDynamic, LinesearchEstimator
import numpy as np

# Use an estimator which tests over the specified epsilon and delta values
n_steps = 1_000
estimator = LinesearchEstimator(
    delta_steps = np.exp(np.linspace(np.log(1e-9), 0, num=n_steps)),  # Log uniform distirbution over [10^-9, 1]
    epsilon_steps = np.linspace(0, 10, num = n_steps)                 # Uniform distribution over [0, 10]
)


# Make a factory corresponding to this search
witness = WitnessDynamic.get_factory(estimator=estimator, confidence_dwk=1, quantiles = np.linspace(0.75, 1, 1_000))
dynamic_run = deltasiege.run("Dynamic", witnesses_factories=witness)

## Plot all Results

Combine all five runs into a single plot. The DP-Opt variation, is not able to dect any violation which can be plotted for this experiment; when selecting the threshold parameter, the resulting $\epsilon_1$ is $-\infty$

In [ ]:
from deltasiege.utils.plot_multiple_trials import plot_all_trials

x_is_delta = False
fig, ax = plot_all_trials(
    (deltasiege_run, "DeltaSiege", "o", "blue"),
    (epsilon_fixed_run, "Fixed $\\delta$", "^", "orange"),
    (delta_fixed_run, "Fixed $\\epsilon$", "x", "purple"),
    (dpsniper_run, "DP-Sniper", "s", "cyan"),
    (dpopt_run, "DP-Opt", "v", "olive"),
    (dynamic_run, "Dynamic", "+", "yellow"),
    x_is_delta=x_is_delta,
)

if x_is_delta:
    ax.set_xticks([delta_0]) 
    ax.set_xticklabels(["$\\delta_0$"], fontsize=20)
    ax.set_yticks([epsilon_0]) 
    ax.set_yticklabels(["$\\epsilon_0$"], fontsize=20)
else:
    ax.set_xticks([epsilon_0]) 
    ax.set_xticklabels(["$\\epsilon_0$"], fontsize=20)
    ax.set_yticks([delta_0]) 
    ax.set_yticklabels(["$\\delta_0$"], fontsize=20)

# Add legend and plot
fig.legend()
fig.show()